# Проект: Исследование стартапов

## Введение

**Цель исследования** состоит в проведении исторического анализа данных о финансировании стартапов для формирования бизнес стратегии в области инвестирования, развития и последующей перепродажи стартапов.  
  
Для анализа представоены данные о с информацией о покупках одними компаниями других компаний, о компаниях и раундах финансирования, о сотрудниках и их образовании, а также дополнительная информация о фондах и раундах инвестирования.  
  
**Задача**  
Подготовить датасет и проверить, что данные в нём соответствуют здравому смыслу, а также ответить на вопросы заказчика:  
- по каким столбцам можно объединять данные из разных таблиц;
- можно ли доверять данным о сотрудниках стартапов и их образовании;
- что значат покупки за 0 или за 1 доллар;
- какая цена в зависимости от категории стартапов и количества раундов финансирования перед его покупкой;
- как рассчитать и отобразить численные показатели и динамику.


## Шаг 1. Знакомство с данными: загрузка и первичная предобработка

Названия файлов:
* acquisition.csv
* company_and_rounds.csv
* people.csv
* education.csv
* degrees.csv

Опциональные датасеты:
* fund.csv
* investment.csv


Они находятся в папке datasets, если вы выполняете работу на платформе. В случае, если вы делаете работу локально, доступ к файлам в папке можно получить по адресу `https://code.s3.yandex.net/datasets/` + имя файла.

### 1.1. Вывод общей информации, исправление названия столбцов

In [ ]:
# Установим модуль phik
!pip install phik

# Загрузим необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# Библиотека понадобится для вычисления моды при агрегации
from scipy import stats
# Библиотека для отключекния предупрждений
import warnings
# Отключение предупреждений
warnings.filterwarnings('ignore')
# Проверка установлен ли phik и его установка, в случае необходимости
try:
    import phik
except ModuleNotFoundError as e:
    !pip install phik
    import phik
    print("Error was:", e)

In [ ]:
# Загрузим необходимые датасеты
# Создадим переменную с путем к файлам
path_to_data = 'https://code.s3.yandex.net/datasets/'


ac_df = pd.read_csv(path_to_data+'acquisition.csv')
cnr_df = pd.read_csv(path_to_data+'company_and_rounds.csv')
peop_df = pd.read_csv(path_to_data+'people.csv')
edu_df = pd.read_csv(path_to_data+'education.csv')
deg_df = pd.read_csv(path_to_data+'degrees.csv')
fund_df = pd.read_csv(path_to_data+'fund.csv')
invest_df = pd.read_csv(path_to_data+'investment.csv')

# Добавим имена для датасетов, чтобы было удобнее в них ориентироваться
ac_df.df_name = 'acquisition'
cnr_df.df_name = 'company_and_rounds'
peop_df.df_name = 'people'
edu_df.df_name = 'education'
deg_df.df_name = 'degrees'
fund_df.df_name = 'fund'
invest_df.df_name = 'investment'

# Создадим список датасетов, чтобы можно было последовательно вызывать их
datasets_list = [ac_df, cnr_df, peop_df, edu_df, deg_df, fund_df, invest_df]

Выведем данные о датасетах и проанализируем результат

In [ ]:
# Ввиду большого количества датасетов напишем фукцию для их проверки
def dataset_view (dataset):
    display(dataset.info())
    display(dataset.head())

In [ ]:
dataset_view(ac_df)

Названия столбцов корректные, типы данных тоже, за исключением столбца `acquired_at`, его необходимо привести к типу `datatime`.

In [ ]:
dataset_view(cnr_df)

Названия столбцов необходимо привести к формату snake_case. При этом необходимо учитывать, что все пробелы в названиях двойные.  
Столбцы `founded at`, `closed at` и `funded at` необходимо привести к формату `datatime`.  
Столбцы `company ID`, `funding round id`, `company id`, `funding  rounds`, `funding  total`, `raised amount` необходимо привести к формату `Int`.  
Столбцы `is first round` и `is last round` необходимо привести к формату `Int` или `bool`.

Отдельные вопросы вызывают столбцы `company  ID` и `company  id`: необходимо понять дублируют ли они друг друга и почему они появились.  
Попробуем это проверить.

In [ ]:
display(cnr_df[['company  ID', 'company  id']].head(10))
display(cnr_df[['company  ID', 'company  id']].tail(10))

Пересечения наблюдаются лишь частично. Продолжим проверку, найдя сочетание солбцов без пропусков.

In [ ]:
display(cnr_df[cnr_df['company  ID'].notnull() & cnr_df['company  id'].notnull()][['company  ID', 'company  id']].head(15))
display(cnr_df[cnr_df['company  ID'].notnull() & cnr_df['company  id'].notnull()][['company  ID', 'company  id']].tail(15))

Мы видим, что в тех строках, где оба столбца заполнены значения совпадают.  
Исходя из этого можно предположить, что подобная ситуация сложилась в результате некорректного соединния двух датасетов и ключи не везде сопоставлены верно. Это надо будет исправить при удалении пропусков.

Приведем названия столбцов к нужному формату. Но перед этим нужно изменить название столбца `company id`. Переименуем его в `company_id_2`

In [ ]:
cnr_df = cnr_df.rename(columns={'company  id': 'company_id_2'})

Так в результате перезаписи датасета произошла замена переменной --- заново присвоим ему имя и запишем в список вместо прежнего

In [ ]:
cnr_df.df_name = 'company_and_rounds'
datasets_list[1] = cnr_df

Произведем замену названий столбцов

In [ ]:
cnr_df.columns = [x.lower().replace('  ', '_') for x in cnr_df.columns.values]
# Выведем результат
cnr_df.info()

In [ ]:
dataset_view(peop_df)

Названия столбцов корректные, данные в столбце `company_id` необходимо привести к формату `Int`

In [ ]:
dataset_view(edu_df)

Названия столбцов корректные. Так как анализ годов выпуска в ходе исследования не планируется, данные в этом столбце можно не изменять.

In [ ]:
dataset_view(deg_df)

Названия столбцов корректные. Данные в столбце `object_id` необходимо исправить, убрав букву перед номером и привести к формату `Int`.

Согласно условиям датасеты `fund_df` и `invest_df` не будут нужны в работе, а требуются лишь для дополнительной информации, но на всякий случай проверим и их. 

In [ ]:
dataset_view(fund_df)

In [ ]:
dataset_view(invest_df)

Датасеты выглядят корректно, но необходима смена тип данных в столбце `founded_at` датасета `fund_df`.

### 1.2. Смена типов и анализ пропусков

- Обработайте типы данных в столбцах, которые хранят значения даты и времени, если это необходимо.
- Оцените полноту данных — сделайте предварительный вывод, достаточно ли данных для решения задач проекта.

Для начала займемся изменением даты. Для этого напишем функцию, куда передадим нужные нам столбцы из нужных датасетов 

In [ ]:
# Функция замены даты
def data_changer(dataset, columns_list):
    for i in columns_list:
        dataset[i] = pd.to_datetime(dataset[i])

Изменим данные в датасетах.

In [ ]:
data_changer(ac_df, ['acquired_at'])
data_changer(cnr_df, ['founded_at', 'closed_at', 'funded_at'])
data_changer(fund_df, ['founded_at'])

Теперь напишем функцию для изменения данных на тип `Int`.

In [ ]:
# Функция изменения типа данных
def int_changer(dataset, columns_list):
    for i in columns_list:
        dataset[i] = dataset[i].astype('Int64')

Передадим туда необходимые данные.

In [ ]:
int_changer(cnr_df, ['company_id', 'company_id_2', 'funding_round_id', 'funding_rounds', 
                     'funding_total', 'raised_amount', 'is_first_round', 'is_last_round'])
int_changer(peop_df, ['company_id'])

Для изменения данных в столбце `object_id` датафрема `deg_df` необходимо сначала удалить оттуда буквенную часть.

In [ ]:
# Уберем буквенную часть
deg_df['object_id'] = deg_df['object_id'].str.replace('p:', '')
# Теперь применим функцию, которая изменит тип данных
deg_df['object_id'] = pd.to_numeric(deg_df['object_id'])

Теперь проверим результат последовательно во всх датафреймах

In [ ]:
for i in datasets_list:
    print(f'Датасет {i.df_name}')
    i.info()
    print('\n')

Теперь типы данных и названия столбцов выглядят корректно.

**Займемся анализом пропусков**

In [ ]:
# Проанализируем пропуски последовательно во всех датасетах
for i in datasets_list:
    print(f'Датасет {i.df_name}')
    print(f'Количество строк в датасете {i.shape[0]}')
    print(f'Количество пропусков в датасете:')
    display(i.isna().agg(['sum', 'mean']).transpose().rename(columns={'sum': 'количество', 'mean': 'доля'}))
    print(f'\n')

Перед началом детального анализа отметим, что удаление пропусков в датасетах является нежелательным, так как они сами отфильтровываются в процессе работы, но при этом их удаление приведет в потере части полезных данных.

***Датасет acquisition*** (`ac_df`): 0.3 % пропусков в `acquired_at` скорее всего говорят об утере данных о дате заключения сделки, однако процент пропусков крайне незначительный и им можно пренебречь. В поле `term_code` --- 80 % пропусков, однако для исследования данная информация не нужна.  
  
  
***Датасет company_and_rounds*** (`cnr_df`): в солбцах `company_id`, `name`, `status`, `investment_rounds`, `funding_rounds`, `funding_total` и `milestones` одинакове количество пропусков. Скорее всего аналогичные пропуски есть и в других столбцах. Корме того, в столбцах `funding_round_id`, `company_id_2`, `funded_at`, `funding_round_type`, `raised_amount`, `pre_money_valuation`, `participants`, `is_first_round` и `is_last_round` также одинаковое количество пропусков. Это хорошо соотносится с гипотизой о том, что данные ошибки возникдли в результате некорректного соединения двух датасетов.
Исправим данные ошибки на следущем этапе.
За исключением этого можно отметить, что в столбцах `domain`, `network_username`, `country_code`, `founded_at` и `closed_at` также присутствует значительное количество пропусков, но для задач анализа данные поля они не нужны.
В поле `category_code` отсутствует порядка 34 % данных. Это значительная цифра и она может сказаться на результатах анализа цены в зависимости от категории стартапов. Однако это надо будет проверить еще раз после дальнейшей обработки датасета.  
  
***Датасет people*** (peop_df): в поле `company_id` доля пропусков составляет 84 %. Это крайне высокое значение и оно критически важно, так как с его помощь можно соотнести сотрудников с компаниями. Такое высокое количество пропусков говорит о том, что получить объективную картину по задействованному в стартапах персоналу не представляется возможным.  
  
***Датасет education*** (edu_df): доля пропусков в столбце `instituition` крайне мала и ей можно пренебречь, а столбец `graduated_at` не является необходимым для исследования.  
  
***Датасет degrees*** (deg_df): отсуствует данные о типе образования 10 % сотрудников, что является не критичным, н довольно значительным. Аналогично отсуствуют данные о спецальности 25 % сотрудников, но это может быть связано с отсуствием образования, подразумевающего получение какой-то специальности.  
  
***Датасет fund*** (fund_df): пропусков в данных, которые могут понадобитсья для исследования нет.  
  
***Датасет investment*** (inv_df): пропуски в данных отсутствуют.

## Шаг 2. Предобработка данных, предварительное исследование


### 2.1. Раунды финансирования по годам

Создадим сводную таблицу на основании столбца raised_amount, где для каждого года указан:
- типичный размер средств, выделяемый в рамках одного раунда;
- общее количество раундов финансирования за этот год.
  
Полученный результат отфильтруем для тех лет, для которых есть информация о более чем 50 раундах финансирования.

In [ ]:
# Создадим сводную таблицу
pivot_cnr = pd.pivot_table(cnr_df, columns=cnr_df['funded_at'].dt.year.astype('Int32'),
                           values=['raised_amount', 'funding_rounds'], 
                           aggfunc={'raised_amount':'median', 'funding_rounds':'count'}).transpose()

# Создадим таблицу, отфильтрованную по строкам, где раундов финансирования больше 50
pivot_cnr_filtered = pivot_cnr[pivot_cnr['funding_rounds'] > 50]

pivot_cnr_filtered

Теперь на основании этой таблицы построим график, отражающий динамику типичного размера средств, которые стартапы получали в рамках одного раунда финансирования.

In [ ]:
plt.figure(figsize=(9,5))
pivot_cnr_filtered['raised_amount'].plot.line(xlabel='Годы', ylabel='Медианный размер финансирования (млн.)', title='Динамика типичного размера средств,\nкоторые стартапы получали в рамках одного раунда финансирования')
plt.show()

На основании полученных результатов видно, что максимальное медианное значение финансирования в рамках одного раунда приходится на 2005 год.  
С 2005 года размер финансирования в раках одного рауна неуклонно снижался, вплоть до 2012 года, после которго показал незначительный рост, что, скорее, может говорить о некоторой стабилизации.  
При этом количество раундов с 1998 года по 2012 год увеличилось в 145 раз --- с 76 до 11 042. Однако в 2013 году впервые за расмариваемых период произошло незначительное снижение числа раундов финансирования, что также может говорть о некоторой стабилизации.  
Интересен тот факт, что данная стабилизация прозошла с в разрезе количества раундов финансирования, и в разрезе объемов выделяемых средств. Можно предположить, что к 2005 году, когда объемы финисирования достигли своего максимум, была пройдена определенная точка, после которой количество раундов и типичный объем выделяемых средств начали демонстрировать обратную зависимость.


### 2.2. Люди и их образование

Заказчик хочет понять, зависит ли полнота сведений о сотрудниках (например, об их образовании) от размера компаний.

- Оцените, насколько информация об образовании сотрудников полна. Используя датасеты `people.csv` и `education.csv`, разделите все компании на несколько групп по количеству сотрудников и оцените среднюю долю сотрудников без информации об образовании в каждой из групп. Обоснуйте выбранные границы групп.
- Оцените, возможно ли для выполнения задания присоединить к этим таблицам ещё и таблицу `degrees.csv`.

Как уже отмечалось, в датасете `people.csv` отсутствует порядка 85 % строк с информацией о компаниях, где работают сотрудники.  
С учетом этого полученные в результате анализа данные будут отличаться весьма низким уровнем достоверности.

Если не принмать в рассчет низкую достоверность данных и работать только с долей записей, где есть информация о компании, то мы получим следующие результаты:

Для начала создадим новый датасет, в котором объединим данные о сотрудниках и их образовании проанализируем его.  
Соединим их по столбцам `id` и `person_id`, использовав левый тип соединения, чтобы сохранить данные обо всех сотрудниках, вне зависимости от наличия сведений об образовании.

In [ ]:
# Создаем новый объединенный датасет
peop_new_df = pd.merge(peop_df, edu_df, left_on='id', right_on='person_id', how='left')

# Выведем данные о новом датасете
print(f'Количетво строк: {peop_new_df.shape[0]}')
display(peop_new_df.isna().agg(['sum', 'mean']).transpose().rename(columns={'sum': 'количество', 'mean': 'доля'}))

Как мы видим информация об образовании отсутствует для 59 % сотрудников. Данные крайне неполные.  
Попробуем добавить к дасету данные из датасета `degrees.csv`. Присоединение будет производить по полям `id_y` и `id`, так как в них совпадают идентификаторы записей об образовании.

In [ ]:
# Добавим данные из еще одного датасета
peop_new_df = peop_new_df.merge(deg_df, left_on='id_y', right_on='id', how='left')

# Выведем информацию о получившемся датасете
print(f'Количетво строк: {peop_new_df.shape[0]}')
display(peop_new_df.isna().agg(['sum', 'mean']).transpose().rename(columns={'sum': 'количество', 'mean': 'доля'}))

Создадим вспомогательный столбец `is_edu`, который будет содержать 1, если о пользователе есть данные в столбце `instituition`, `degree_type` или `subject`.

In [ ]:
# Функция, возвращающая единицу или ноль в зависимости от наличия данных об образовании хотя бы в одном столбце
def is_edu (data):
    if data['instituition'] is not np.nan:
        return 1
    elif data['degree_type'] is not np.nan:
        return 1
    elif data['subject'] is not np.nan:
        return 1
    else:
        return 0

# Создадим новый столбец, применив данную функцию
peop_new_df['is_edu'] = peop_new_df.apply(is_edu, axis=1)

# Выведем информацию об обновленном датасете
peop_new_df.info()
display(peop_new_df.isna().agg(['sum', 'mean']).transpose().rename(columns={'sum': 'количество', 'mean': 'доля'}))

Мы видим, что доля записей об образование такая же --- 59 %, а значит присоединение датафрейма `degrees.csv` не добавило информации.

Для дальнейшей работы исключим все записи о сотрудниках, где отсутствует идентификатор компании, так как без него мы не сможем определить относятся ли сотрудники к одной компании ли к разным.

In [ ]:
peop_new_df = peop_new_df.dropna(subset=['company_id'])
peop_new_df.info()

# Выведем информацию о том, какой процент датасета сохранился
print('\nДоля нового датасета от старого:', round(peop_new_df.shape[0]/267694, 2))

Осталось всего 44 789 записей.  
Теперь посчитаем количество сотрудников в разных компаниях и найдем максимальное значение и медиану.

In [ ]:
# Сгруппируем данные по компаниям, чтобы посчитать количество сотрудников в каждой компании
peop_comp = peop_new_df.groupby('company_id').agg({'id_x': 'count'})

# Выведем описание данных
peop_comp.describe()

Медианное количество сотрудников в компании --- 1, в 75 % компаний --- не больше 2.  
С такими данными деление компаний на группы и дальнейший анализ не представляется целесообразным.

В подтверждение этого построим диаграмму распределения.

In [ ]:
plt.figure(figsize=(12, 6))
plt.boxplot(peop_comp.reset_index()['id_x'], vert=False)
plt.show()

Разброс настолько сильный, что часть без выбросов на графике почти не видна.

Попробуем понять как много сотрудников в каких категориях. Сгруппируем данные, но на это раз по количеству сотрудников

In [ ]:
# Сбросим индексы
peop_comp = peop_comp.reset_index()

In [ ]:
# Провреим сколько сотрудников в компаниях какого размера
peop_comp.groupby('id_x').count()


Разделим компании на категории. Наибольшее количество компаний - с одним сотрудником, их выделим в отдельную категорию. Компаний с двумя или тремя сотрудниками тоже довольно много, их тоже можно выделить в самостоятельные категории. Компании из 4-5 человек можно объединить в одну категорию, чтобы приблизить ее по количеству человек к предыдущим. Остальные компании разделим на группы  6-10 человек, 11-15 человек и больше 25 человек.  
Разделим датафрейм на соответствующие группы и посчитаем количество сотрудникво в данных группах.

In [ ]:
bins   = [0, 1, 2, 3, 5, 10, 25, float('inf')]
labels = ['1', '2', '3', '4-5', '6-10', '11-25', '26+']
pd.cut(peop_comp['id_x'], bins=bins, labels=labels).value_counts()

Полученный результат выглядит удовлетворительно, будем работать с ним.

Присвоим категории данным в датафрейме.

In [ ]:
peop_comp['category'] = pd.cut(peop_comp['id_x'], bins=bins, labels=labels)

Во избежании потери данных об образовании присоединим категории к датафрейму peop_new_df по индексу компании

In [ ]:
# Соединяем датафреймы датафреймы по принципу "многие к одному"
peop_new_df = peop_new_df.merge(peop_comp[['company_id', 'category']], on='company_id', how='left', validate='m:1')

Теперь используя данные категории построим таблицу и график с процентным соотношением наличия данных об образовании сотрудников. 

In [ ]:
peop_new_df.groupby('category').agg(count=('id_x', 'count'), share=('is_edu', 'mean'))

In [ ]:
sns.countplot(data=peop_new_df, x='category', hue='is_edu')
plt.show()

Как мы видим, если в компании больше одного человека, то вероятность того, что информаци об образовании присутствует существенно выше

### 2.3. Объединять или не объединять — вот в чём вопрос

Некоторые названия столбцов встречаются в датасетах чаще других. В результате предварительной проверки датасетов было выяснено, что столбец `company_id` подходит для объединения данных.

- Установите, подходит ли для объединения данных столбец `network_username`, который встречается в нескольких датасетах. Нам необходимо понимать, дублируется ли для разных датасетов информация в столбцах с таким названием, и если да — то насколько часто.
- Оцените, можно ли использовать столбцы с именем `network_username` для объединения данных.

Столбец `network_username` встречается в двух датасетах: `company_and_rounds` и `people`.  
Учитывая, что нам известно о том, что данные в столбце `company_id` подходят для объединения и также встречаются в обоих датасетах, можно объяединить датасеты по столбцу `company_id` и проверить совпадают ли в полученных строках данные в столбцах `network_username` и как часто.  
  
Объединим датасеты. Используем внутреннее соединение, чтобы у нас остались только совпадающие значения.

In [ ]:
cnr_merge_df = cnr_df[['company_id', 'network_username']]
cnr_merge_df.info()
peop_merge_df = peop_df[['company_id', 'network_username']]
peop_merge_df.info()
user_merge_df = pd.merge(cnr_merge_df, peop_merge_df, on='company_id', how='inner')
user_merge_df.info()
#user_merge_df.head()

Для решения задачи сформируем два списка с наименованиями из `network_username` и сравним их между собой.

In [ ]:
# Найдем уникальные значения в cnr_network
cnr_network = cnr_df['network_username'].unique()
cnr_df_count = cnr_df['network_username'].count()
print('Всего записей в network_username из датасета company_and_rounds:', cnr_df_count)
print(f'Количество уникальных записей в network_username из датасета company_and_rounds: {cnr_network.shape[0]}')
print('Количество дубликатов записей в network_username из датасета company_and_rounds:', cnr_df_count - cnr_network.shape[0])
print('Количество пропусков в network_username из датасета company_and_rounds:', cnr_df['network_username'].isna().count())
print('\n')
# Найдем уникальные значения в peop_network
peop_network = peop_df['network_username'].unique()
peop_df_count = peop_df['network_username'].count()
print('Всего записей в network_username из датасета people:', peop_df_count)
print(f'Количество уникальных записей в network_username из датасета people: {peop_network.shape[0]}')
print('Количество дубликатов записей в network_username из датасета people:', peop_df_count - peop_network.shape[0])
print('Количество пропусков в network_username из датасета people:', peop_df['network_username'].isna().count())

# Посчитаем долю таких значений
print(f'\nСоотношение размеров датафреймов друг к другу:', round(peop_network.shape[0]/cnr_network.shape[0], 2))

Количество строк разное, но надо сравнить и сами значения, выяснив, сколько среди них есть пересечений.

In [ ]:
# Создадим переменную счетчик
net_unite = list(set(cnr_network) & set(peop_network))
print(f'Количество пересечений в двух списках: {len(net_unite)}')
print(f'Доля пересечений в датасете company_and_rounds: {round(len(net_unite)/cnr_network.shape[0], 2)}')
print(f'Доля пересечений в датасете people: {round(len(net_unite)/peop_network.shape[0], 2)}')

Значения повторяются всего 2200 раз, а доля пересечений крайне мала. Это свидетествует о том, что использовать данные столбцы для объединения датасетов не получится.


### 2.4. Проблемный датасет и причина возникновения пропусков

Во время собственного анализа данных у заказчика больше всего вопросов возникло к датасету `company_and_rounds.csv`. В нём много пропусков как раз в информации о раундах, которая заказчику важна.

- Любым удобным способом приведите данные в вид, который позволит в дальнейшем проводить анализ в разрезе отдельных компаний. Обратите внимание на структуру датасета, порядок и названия столбцов, проанализируйте значения.

По гипотезе заказчика данные по компаниям из этой таблицы раньше хранились иначе, более удобным для исследования образом.

- Максимальным образом сохраняя данные, сохранив их связность и исключив возможные возникающие при этом ошибки, подготовьте данные так, чтобы удобно было отобрать компании по параметрам и рассчитать показатели из расчёта на одну компанию без промежуточных агрегаций.

Как уже отмечалось ранее ряд факторов --- в том числе название --- указывает на то, что датасет сформирован путем объединения двух датасетов, но судя по всему, объединения некорректного.  
Выведем информацию о нем еще раз.

In [ ]:
cnr_df.info()
display(cnr_df.isna().agg(['sum', 'mean']).transpose().rename(columns={'sum': 'количество', 'mean': 'доля'}))

Скорее всего столбцы `funding_round_id`, `company_id_2`, `funded_at`, `funding_round_type`, `raised_amount`, `pre_money_valuation`, `participants`, `is_first_round` и `is_last_round` ранее представляли собой отдельный датасет.  
Разделим их на два отдельный датафрейма.

In [ ]:
# Создадим датафрейм company_df, выделив нужные столбцы из общего датафрейма и удалим все пустые строки и дубликаты в нем
company_df = cnr_df[['company_id', 'name', 'category_code', 'status', 'founded_at', 'closed_at', 'domain', 
                     'network_username', 'country_code', 'investment_rounds', 'funding_rounds', 
                     'funding_total', 'milestones']].dropna(how='all').drop_duplicates()
company_df.info()

Проверим все ли значения `company_id` теперь уникальны.

In [ ]:
company_df.shape[0] == company_df['company_id'].unique().shape[0]

Проделаем тоже самое, создав датафрейм `rounds_df`.

In [ ]:
rounds_df = cnr_df[['funding_round_id', 'company_id_2', 'funded_at', 'funding_round_type', 'raised_amount', 
                 'pre_money_valuation', 'participants', 'is_first_round', 'is_last_round']].dropna(how='all').drop_duplicates()
rounds_df.info()

Для удобства поменяем название столбца `company_id_2` на `company_id`.

In [ ]:
rounds_df = rounds_df.rename(columns={'company_id_2': 'company_id'})

Проверим все ли значения `funding_round_id` уникальны.

In [ ]:
rounds_df.shape[0] == rounds_df['funding_round_id'].unique().shape[0]

Сделаем то же самое для `company_id`.

In [ ]:
display(rounds_df.shape[0] == rounds_df['company_id'].unique().shape[0])
print(f'Количество ункальных значений:', rounds_df['company_id'].unique().shape[0])

Как и ожидалось идентификаторов компаний меньше, чем идентификаторов раундов, так как некоторые компании проходили несколько раундов.  
  
Теперь по просьба заказчика необходимо собрать данные воедино так,  чтобы удобно было отобрать компании по параметрам и рассчитать показатели из расчёта на одну компанию без промежуточных агрегаций.  
  
Проблема заключается в том, что собрать датасет так, чтобы он не требовал промежуточных агрегаций по компании, но при этом сохранил все данные не получится: у некоторых компаний было несколько раундов финансирования и простое объединение детафреймов приведет к задвоению данных по некоторым компаниям. При этом, если сгруппировать данные заранее, то мы потеряем информацию об отдельных раундах финансирования.  
  
Так как заказчик ориентируется в задании на рассчет данных по одной компании, то пожертвуем информацией о раундах финансирования.  
  
Для этого возьмем за основу датасет `company_df`, чтобы избежать промежуточных агрегаций в дальнейшем датасет `rounds_df` заранее сгруппируем по столбцу `company_id`, а затем присоеденим к `company_df`.  
  
Переименуем часть столбцов под новые значения и вместо столбца `funded_at` добавим два столбца `first_founding` и `last_founding` с датами первого и последнего финансирования. Для типа финансировая найдем моду, как наиболее часто использованный тип.

Столбец `raised_amount` дублируется в `funding_total`, а количество раундов финансирования содержится в столбце `funding_rounds`. Их можно опустить.

In [ ]:
# Сгруппируем датасет rounds_df
rounds_group = rounds_df.groupby('company_id').agg(first_founding=('funded_at', 'min'), last_founding=('funded_at', 'max'), 
                                                   funding_type=('funding_round_type', lambda x: stats.mode(x)[0]), 
                                                   pre_money_valuation=('pre_money_valuation', 'first'), 
                                                   participants=('participants', 'sum')).reset_index()

# Объединим два датасета
cnr_new_df = company_df.merge(rounds_group, left_on='company_id', right_on='company_id', how='left', validate='1:m')

cnr_new_df.info()
cnr_new_df.head()

Успешно сформирован новый датасет, с которым можно работать без дополнительных агрегаций и где данные соеденины корректно. Информация о раундах фининсирования добавлена для тех фирм, где она иммется в наличии.


## Шаг 3. Исследовательский анализ объединённых таблиц


### 3.1. Объединение данных

Объедините данные для ответа на вопросы заказчика, которые касаются интересующих его компаний. Заказчика прежде всего интересуют те компании, которые меняли или готовы менять владельцев. Получение инвестиций или финансирования, по мнению заказчика, означает интерес к покупке или продаже компании.

В качестве основы для объединённой таблицы возьмите данные из обработанного датасета `company_and_rounds.csv` — выберите только те компании, у которых указаны значения `funding_rounds` или `investment_rounds` больше нуля, или те, у которых в колонке `status` указано `acquired`. В результирующей таблице должно получиться порядка 40 тысяч компаний.

Проверьте полноту и корректность получившейся таблицы. Далее работайте только с этими данными.

Воспользуемся полученным на прошлом этапе датафреймом и на его снове создадим новый отфильтрованный датафрейм.

In [ ]:
crn_buying = cnr_new_df[(cnr_new_df['funding_rounds'] > 0) | (cnr_new_df['investment_rounds'] > 0) | 
                                (cnr_new_df['status'] == 'acquired')]
crn_buying.info()

Убедимся, что среди идентификаторов компаний нет дубликатов.

In [ ]:
crn_buying['company_id'].duplicated().sum()


### 3.2. Анализ выбросов

Заказчика интересует обычный для рассматриваемого периода размер средств, который предоставлялся компаниям.

- По предобработанному столбцу `funding_total` графическим способом оцените, какой размер общего финансирования для одной компании будет типичным, а какой — выбивающимся.
- В процессе расчёта значений обратите внимание, например, на показатели, возвращаемые методом `.describe()`, — объясните их. Применимы ли к таким данным обычные способы нахождения типичных значений?

In [ ]:
plt.figure(figsize=(15,5))
# Так как выбросы очень существенные - построим диаграмму размаха исключив их
crn_buying['funding_total'].plot(kind='box', vert=False, grid=True, showfliers=False)
plt.show()

Типачный размер финансирования для одной компании будет приблизительно около 800 тысяч. Размер фининсирования больше 1,4 млн. будет выбивающимся.

In [ ]:
crn_buying['funding_total'].describe()

Для уменьшения влияния выбросов и разницы значений на визуальное представление данных, построим гистограмму с логарифмической шкалой.

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(crn_buying['funding_total'], bins=50, edgecolor='black', log=True)
plt.xscale('log')  # Логарифмическая шкала по оси X
plt.xlabel('Общее финансирование (логарифмическая шкала)')
plt.ylabel('Количество компаний')
plt.title('Распределение общего финансирования (логарифмическая шкала)')
plt.show()

В данных наблюдается очень большой разброс и сильные выбросы. Около 25 % Данных - нулевые значения. Из-за сильного перекоса в данных стандартное отклонение в пять раз больше среднего значения. С такими данными очень сложно определить типичные значения и рекомендуеся их предварительно очистить. Вместе с тем отдельно можно провести анализ выбросов, для выявления интересных тенденций.  
  
В качестве рекомендации стоит проверить источник данных и определить являются ли нулевые значения реальными показателями или это ошибка в данных.  
  
Кроме того, принимая по внимание большой временной диапазон представленных данных, рекомендуется провести анализ типичных значений с учетом временного среза, так как данные могут сильно зависить от инфляции и других временнЫх тенденций, которые в настоящее время потеряли свою актуальность.


### 3.3. Куплены забесплатно?

- Исследуйте компании, которые были проданы за ноль или за один доллар, и при этом известно, что у них был ненулевой общий объём финансирования.

- Рассчитайте аналитически верхнюю и нижнюю границу выбросов для столбца `funding_total` и укажите, каким процентилям границы соответствуют.

Создадим датафрейм, объединяющий данные о компаниях и данные о покупках

In [ ]:
ac_company_df = crn_buying.merge(ac_df, left_on='company_id', right_on='acquired_company_id', how='left')
ac_company_df.info()

Проверим, что стоимость покупки компаний не была отрицательной.

In [ ]:
ac_company_df[(ac_company_df['price_amount'] < 0)]

Таких данных нет. Это поможет при дальнейшей фильтрации.

Теперь отфильтруем датафрейм по заданным условиям и выведем по нему информацию.

In [ ]:
#Фильтрация и создание нового датаврейма
ac_filtered_df = ac_company_df[(ac_company_df['price_amount'] <= 1) & (ac_company_df['funding_total'] > 0)]

#КОличество записей
print('Количество записей о подобных компаниях:', ac_filtered_df.shape[0])

#Описание данных
ac_filtered_df[['investment_rounds', 'funding_rounds', 'funding_total', 'pre_money_valuation', 'participants', 'price_amount']].describe()

На основании полученных данных можно сказать, что у почти у всех подобных компаний не было **ни одного раунда инвестирования** и, чаще всего, **один или два раунда финансирования**.  
**Типичный размер финисирования составлял 6 млн.**  
Чаще всего **предварительная оценка таких компаний является нулевой.  
Типичное число участников --- 3.  
Сумма сделки во всех случаях нулевая**.

Рассчитаем верхнюю и нижнюю границу выбросов для столбца funding_total у указанных компаний.

Посчитаем межквартильный размах (IQR) столбца `funding_total`, найдем с его помощью выбросов.

In [ ]:
# Посчитаем межквартильный размах
ft_iqr = ac_filtered_df['funding_total'].quantile(0.75) - ac_filtered_df['funding_total'].quantile(0.25)

#Найдем верхнюю и нижнюю границу выбросов
low_outlier = ac_filtered_df['funding_total'].quantile(0.75)+1.5*ft_iqr
hight_outlier = ac_filtered_df['funding_total'].max()

# Выведем их
print('Нижняя граница выбросов -', low_outlier)
print('Верхняя граница выбросов -', hight_outlier)

Рассчитаем какому процентилю соответствует нижняя и верхняя границы

In [ ]:
print(f"Нижняя граница выбросов соответствует {ac_filtered_df[ac_filtered_df['funding_total'] < low_outlier]['funding_total'].count()/ac_filtered_df['funding_total'].count()*100:.2f} процентилю")
print(f"Верхняя граница выбросов соответствует {ac_filtered_df[ac_filtered_df['funding_total'] < hight_outlier]['funding_total'].count()/ac_filtered_df['funding_total'].count()*100:.3f} процентилю")

Удостоверимся в отсутствии отрицательных значений.

In [ ]:
ac_filtered_df[ac_filtered_df['funding_total'] < 0]

Отрицательные значения отсутствуют, все корректно.


### 3.4. Цены стартапов по категориям

Категории стартапов с типично высокими ценами покупки стартапов и значительным разбросом цен могут быть привлекательными для крупных инвесторов, которые готовы к высоким рискам ради потенциально больших доходов. Среди категорий стартапов выделите категории стартапов, характеризующиеся:

- типично высокими ценами;
- и наибольшим разбросом цен за стартап.

Объясните, почему решили составить топ именно из такого числа категорий и почему рассчитывали именно так.

Для начала выясним сколько категорий стартапов у нас есть.

In [ ]:
ac_company_df['category_code'].nunique()

Топ-10  --- хороший варинат для представления данных. Он составляет почти четверть данных и при этом круглое число хорошо воспринимается людьми.  
  
Сгруппируем данные по категориям и найдем для их цен медиану, как типичное значение и стандартное отклонение, как показатель разброса.

In [ ]:
category_df = ac_company_df.groupby('category_code').agg(median=('price_amount', 'median'), volatility = ('price_amount', 'std'))

Отсортируем данные по медианной цене и выведем топ-10

In [ ]:
category_df['median'].sort_values(ascending=False).head(10)

10 значений не набирается, выведем топ-5

In [ ]:
print('Топ-5 категорий по типичным ценам продажи')
category_df['median'].sort_values(ascending=False).head(5)

Для более симмитричного представления данных сформируем топ-5 по разбросу цен продажи.

In [ ]:
print('Топ-5 категорий по высокому разбросу цен продажи')
category_df['volatility'] = category_df['volatility'].round(2)
print(category_df['volatility'].sort_values(ascending=False).head(5))


### 3.5. Сколько раундов продержится стартап перед покупкой

- Необходимо проанализировать столбец `funding_rounds`. Исследуйте значения столбца. Заказчика интересует типичное значение количества раундов для каждого возможного статуса стартапа.
- Постройте график, который отображает, сколько в среднем раундов финансирования проходило для стартапов из каждой группы. Сделайте выводы.

Для начала отфильтруем те компании, к уоторых не было ни одного раунда финансирования.

In [ ]:
crn_rounds_df = cnr_new_df[cnr_new_df['funding_rounds'] > 0]

Выведем информацию о `funding_rounds` в зависимости от статуса стартапа.

In [ ]:
crn_rounds_df = crn_rounds_df.groupby('status')['funding_rounds'].agg('describe')
crn_rounds_df

Разброс в данных получился достаточно небольшой, кроме того, можно сказать, что данные довольно однородны: **медианное количество раундов финансирования все всех статусов, кроме ipo, составляет 1 раунд, а для ipo --- 2. Наибольший разброс по количеству раундов наблюдается у стартапов со статусом ipo, наименьший --- у закрытых.  
Стоит отметить, что хароактеристики раундов финансировния у статусов acquired и operating довольно близки, хотя количество таких статусов различается в 10 раз в пользу действующих компаний**

Построим график, отображающий среднее количество раундов финансирования в зависимости от статуса. Лучше всего тут подойдет столбчатая диаграмма.

In [ ]:
plt.figure(figsize=(7,5))
crn_rounds_df.reset_index().sort_values(by='mean').plot(kind='barh', x='status', y='mean', legend=False, xlabel='Статус стартапа', title='Среднее количество раундов финансирования')
plt.show()


## Шаг 4. Итоговый вывод и рекомендации

Опишите, что было сделано в проекте, какие были сделаны выводы, подкрепляют ли они друг друга или заставляют сомневаться в полученных результатах.  
  
Датасеты загружены, проанализированны, данные в них приведены к нужным типам, названия приведены к корректному формату, исправлены ошибки, проанализированы пропуски.  
**Анализ раундов финансирования** по годам показал, что пиковая типичная сумма финансирования приходится на 2005 год, далее наблюдается спад сумм финансирования и рост числа раундов финансирования, со стабилизацией в 2012 году.
**Анализ датасетов, содержащих сведения об образовании сотрудников** показал, что данные крайне неполные, нерепрезентативные и не подходят для корректного анализа. Однако, в соответствии с заданием, был проведен анализ имеющихся сведений на предмет выявления взаимосвязи размера компании с наличием сведений об образовании сотрудников. Анализ показал, что в компаниях из двух и более человек выше вероятность наличия сведений об образовании.  
**Сравнение данных в столбцах `network_username`** разных датасетов показал, что этот столбец содержит разные сведения и не подходит для объединения датасетов.  
**Анализ датасета `company_and_rounds`** показал, что он получен в результате некорректного соединения двух датасетов. Датасет был разделен на два датасета для корректной работы. На основе этих датасетов по заданию заказчика был сформирован датасет, позволяющий получать информацию о компаниях и раундах финансирования без дополнительных агрегаций.  
**Исследовательский анализ данных** показал, что типачный размер финансирования для одной компании составляет приблизительно  800 тысяч, а размер фининсирования больше 1,4 млн. будет попадать в категорию выбросов.  
**Анализ компаний, купленых за 0 или 1 доллар, но имеющих ненулевой объем финансирования** показал, что почти у всех подобных компаний не было ни одного раунда инвестирования и, чаще всего, один или два раунда финансирования. Типичный размер финисирования составлял 6 млн. Выбросы в в размерах итоговых фининсирования в данной категории начинаются после 90-ого процентиля.  
**Наиболее высокая типичная стоимость** у компаний отнощихся к категориям nanotech, manufacturing, real_estate, automotive, biotech. **Наибольший разброс стоимости** наблюдается внутри категорий enterprise, public_relations, real_estate, automotive, biotech.  
**Анализ раундов финансирования стартапов перед покупкой** показал, что медианное количество раундов финансирования все всех статусов, кроме ipo, составляет 1 раунд, а для ipo --- 2 раунда. Наибольший разброс по количеству раундов наблюдается у стартапов со статусом ipo, наименьший --- у закрытых.  
Стоит отметить, что **характеристики раундов финансировния у статусов acquired и operating довольно близки**, хотя количество таких статусов различается в 10 раз в пользу действующих компаний.

#### Рекомендации
  
В виду ограниченных данных об образовании сотрудников, рекомендуется принимать их в рассчет с крайней осторожность или провести повторное исследование, с уточненными данными.  
  
Так как практически у всех компаний, купленных за 0 долларов не было ни одного раунда инвестирования, а только раунды финансирования, можно предположить, что последущий переход права собственности был одним из условий финансирования, делающее его опхожим на определенную форму инвестирования. Это можно учитывать, как один из механизмов инвестирования в компании, но для более подробной рекомендации стоит проработать юридические аспекты подобных сделок.  
  
Для более надежного инвестирования рекомендуются компании из категорий nanotech так как типичная цена их последующей продажи наиболее высокая.  
Если инвестиционная стратегия более рисковая, то рекомендуется обратить внимание на компании категории enterprise, так там наблюдается наибольший разброс цен, а следовательно больше шансов найти компанию, которая может показать довольно большой рост. Но такая стратегия требует осторожного и глубокого анализа.  
  
Типичное количество раундов финансирования перед покупкой компании --- 1, для всех стартапов, кроме ipo, однако самих таких компаний крайне мало. Это говорит о том, что получить такой статус довольно сложно, но он вызывает доверие у тех, кто готов вложить в компанию средства. Исходя из этого можно использовать стратегию преобретения компаний в статусе ipo с одним или меньше раундами финансирования или перевода компании, в которую вложены инвестиции, в статус ipo для получения дополнительного финансирования в будущем.